# DRIN Preprocessing Script1 - SMHI climate projections

This notebook is intended to process the climate projection data from the Swedish Metrological and Hydrological Institute (SMHI) in order to develop water flow in the Drin river segments. 

Note: To be able to run this notebook you you have the required raw data under 'data/smhi_data'.

### Import packages and dependencies. 

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
import datetime as dt
import glob
import re
from pathlib import Path
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

# Part 1: Historical flow (Reference Scenario)

### 1.1 Seting up the dataframe 

In [3]:
# Specify the folder and the scenario path

folder = os.path.join('..','Data', 'smhi_data')
scenario_path = os.path.join(folder ,'ref')


In [4]:
# reading the 'raw_*.csv', performing some changes and saving it as 'pros_*.csv':

for filepath in glob.iglob(os.path.join(scenario_path, ('raw_*.csv'))):
    #print(filepath)
    df=pd.read_csv(filepath)
    i=filepath[26:31]
    df['catchment']= i
    df['scenario']= 'REF'
    df['model']='e_hype'
    df.rename({'River discharge (m3/s)':'value'}, axis=1, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Date'] = pd.to_datetime(df["Date"].dt.strftime('%d/%m/%Y'))
    df=df[['model','catchment','scenario','Date','value']]
    
    ##print(i)
    #df.to_csv(os.path.join(scenario_path, 'pros_' + str(i) + '.csv'))

In [5]:
# merging the 'pros_*.csv' files:
joined_files = os.path.join(scenario_path, "pros_*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined in one dataframe:
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

df2 = pd.pivot_table(df, values='value', index=['Date'],
                    columns=['catchment'], aggfunc=np.sum).round(3).reset_index()

# Save the file:
#df2.to_csv(os.path.join(scenario_path,'all_catchments_processed.csv'))


In [6]:
df2.describe()

catchment,black,fierz,globo,koman,ohrid,spilj,vaudj,white
count,10957.000000,10957.000000,10957.000000,10957.000000,10957.000000,10957.000000,10957.000000,10957.000000
mean,78.099397,169.861543,18.124613,199.440866,5.025431,52.685578,236.833895,56.108966
std,83.821065,143.619135,16.230676,163.803115,6.440248,51.220808,189.099657,56.459141
min,0.000000,0.000000,0.355000,0.000000,0.000000,1.109000,0.000000,2.136000
25%,21.041000,140.066000,6.311000,140.280000,0.497000,19.280000,136.779000,17.956000
50%,51.526000,140.066000,16.897000,159.733000,2.108000,36.611000,186.007000,38.681000
75%,104.698000,185.692000,17.611000,237.145000,7.814000,69.990000,304.418000,72.980000
max,846.617000,1208.006000,165.939000,1350.018000,39.646000,462.182000,1518.558000,522.583000


### NOTE: 

Checking the dataset shows that the max values occured in these years: 1985, 1986, 2004 and 2010
we know that the year 2010 was a flooding years in the region, therefore we will remove this year (2010) from the calculations

Fierza max in 2010
black max in 1985
globo max in 1986
koman max in 2010
ohrid max in 1986
spilj max in 1985
vaudj max in 2010
white max in 2004


### 1.2 Restructuring the river segments and introducing the missing catchments

In [7]:
# Introducing the missing catchments: (This is related to the structure of the hydrological system in osemosys, which is lightly different than the structure in smhi portal)

df2 = df2[['Date', 'ohrid','globo', 'spilj','black', 'white', 'fierz', 'koman', 'vaudj']]

df2.rename({
#'ohrid':'ohrid_ct',  #ohrid lake outflow
#'globo':'BDCT1',      #Inflow from Golbo catchment
'globo':'BDRS1',      #Inflow from Golbo catchment
'spilj':'BDRS2',      #Inflow in river segment before spilje
'black':'BDRS3',      #Inflow in the river segment after spilje and before white drin and skavica
'white':'WDRS1',      #Inflow from white drin
'fierz':'DRS1',       #Inflow in river segment before fierza HPP
'koman':'DRS2',       #Inflow in river segment before koman HPP
'vaudj':'DRS3'        #Inflow in river segment before Vau i dejes HPP
}, axis=1, inplace=True)

#df2['BDRS1'] = df2['ohrid']+df2['BDCT1']  #Inflow in river segment before Globocica
df2.loc[df2['BDRS1']>df2['ohrid'], 'BDCT1'] = df2['BDRS1']- df2['ohrid']
df2.loc[df2['BDRS1']<=df2['ohrid'], 'BDCT1'] = df2['BDCT1'].min()

#Inflow from catchment after Globocica and before Spilje
df2.loc[df2['BDRS2']>df2['BDRS1'], 'BDCT2'] = df2['BDRS2']-df2['BDRS1']
df2.loc[df2['BDRS2']<=df2['BDRS1'], 'BDCT2'] = df2['BDCT2'].min()

#Inflow from catchment after Spilje and before white drin 
df2.loc[df2['BDRS3']>df2['BDRS2'], 'BDCT3'] = df2['BDRS3']-df2['BDRS2']
df2.loc[df2['BDRS3']<=df2['BDRS2'], 'BDCT3'] = df2['BDCT3'].min()


#Inflow from catchment before Fierza 
df2.loc[df2['DRS1']>(df2['BDRS3']+df2['WDRS1']), 'DCT1'] = df2['DRS1']-(df2['BDRS3']+df2['WDRS1'])
df2.loc[df2['DRS1']<=(df2['BDRS3']+df2['WDRS1']), 'DCT1'] = df2['WDRS1'].min()

#Inflow from catchment before Koman
df2.loc[df2['DRS2']>df2['DRS1'], 'DCT2'] = df2['DRS2']-df2['DRS1']
df2.loc[df2['DRS2']<=df2['DRS1'], 'DCT2'] = df2['DCT2'].min()

#Inflow from catchment before vau i dejes
df2.loc[df2['DRS3']>df2['DRS2'], 'DCT3'] = df2['DRS3']-df2['DRS2']
df2.loc[df2['DRS3']<=df2['DRS2'], 'DCT3'] = df2['DCT3'].min()
            

df2 = df2[['Date','ohrid', 'BDCT1','BDRS1','BDCT2', 'BDRS2','BDCT3', 'BDRS3', 'WDRS1', 'DCT1','DRS1','DCT2', 'DRS2','DCT3','DRS3']]



### 1.3 Processing smhi reference scenario data to generate timeseries flows

#Organizing the time series data of historical flow for the visulaization: 


In [10]:
# To generate a timeseries dataset:

dfts= df2.copy()
dfts['scenario'] = 'ref'
dfts['year'] = pd.DatetimeIndex(dfts['Date']).year
dfts = dfts.loc[dfts['year']!=2010]  # To remove the year 2010
dfts['model']='ehype'
dfts = dfts.melt(id_vars=['Date', 'year','model', 'scenario'], var_name='catchment',value_name='value',ignore_index=True)

# Save:
output_folder = os.path.join('processed_data','smhi_ref_data')
os.makedirs(output_folder, exist_ok=True)
#dfts.to_csv(os.path.join(output_folder, 'ehype_ref_timeseries_allcatchment_b20220211.csv'))

### 1.4 Processing smhi reference scenario data to generate weekly flows

In [11]:
# Aggregate by week and use 2020 as the reference year:

dff = df2

dff['year'] = pd.DatetimeIndex(dff['Date']).year
dff['month']=pd.DatetimeIndex(dff['Date']).month
dff['week']=pd.DatetimeIndex(dff['Date']).week
dff['day']=pd.DatetimeIndex(dff['Date']).day
dff=dff.loc[dff['year']!=2010]
dff=dff.groupby('week').mean().round(3).reset_index()
dff.drop(['year','month','day'], axis=1, inplace=True)
dff['model']='ehype'
dff['scenario']='ref'
dff['year']='2020'

dff_melted = dff.melt(id_vars=['model','scenario','year','week'], var_name='catchment',value_name='value',ignore_index=True)

dff_melted = dff_melted[['model','scenario','year','catchment','week','value']]

# Saving the output file

results_folder = os.path.join('processed_data', 'smhi_ref_data')
os.makedirs(results_folder, exist_ok = True)
#dff_melted.to_csv(os.path.join(results_folder, 'ehype_ref_weeklyavg_allcatchments_b20220211.csv'), index=False)


<ipython-input-11-77eaa53666a2>:7: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dff['week']=pd.DatetimeIndex(dff['Date']).week


In [13]:
dff_melted

,model,scenario,year,catchment,week,value
0,ehype,ref,2020,ohrid,1,6.420
1,ehype,ref,2020,ohrid,2,3.224
2,ehype,ref,2020,ohrid,3,6.018
3,ehype,ref,2020,ohrid,4,6.617
4,ehype,ref,2020,ohrid,5,7.221
...,...,...,...,...,...,...
737,ehype,ref,2020,DRS3,49,226.816
738,ehype,ref,2020,DRS3,50,251.407
739,ehype,ref,2020,DRS3,51,320.877
740,ehype,ref,2020,DRS3,52,329.575


### 1.5 Generating flow graphs

### Timeseties flow graphs:


In [17]:
# 'ohrid', 'BDCT1', 'BDRS1', 'BDCT2', 'BDRS2', 'BDCT3', 'BDRS3','WDRS1', 'DCT1', 'DRS1', 'DCT2', 'DRS2', 'DCT3', 'DRS3'

river_segments = ['ohrid', 'BDCT1', 'BDRS1', 'BDCT2', 'BDRS2', 'BDCT3', 'BDRS3','WDRS1', 'DCT1', 'DRS1', 'DCT2', 'DRS2', 'DCT3', 'DRS3']

for river_segment in river_segments:
    dff = dfts.loc[dfts['catchment']==river_segment]
    graph_title = river_segment + ' historical flow' + ' -Reference Scenario'
    fig = px.line(dff, x='Date', y='value', color='model',
                  labels={"value": "River discharge (m3/sec)"}, title=graph_title,
                  facet_col_spacing=0.05)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    #You can change the image extension to *.png if you want or keep it as pdf (for high resolution)
    output_folder = os.path.join('processed_data','smhi_ref_graphs','Timeseries')
    os.makedirs(output_folder, exist_ok = True)

    #fig.write_image('processed_data/smhi_flow_graphs/REF/Timeseries/{}.png'.format(graph_title))
    #fig.show()
   

### Weekly flow graphs

In [15]:
# 'ohrid', 'BDCT1', 'BDRS1', 'BDCT2', 'BDRS2', 'BDCT3', 'BDRS3','WDRS1', 'DCT1', 'DRS1', 'DCT2', 'DRS2', 'DCT3', 'DRS3'

river_segments = ['WDRS1', 'DCT1', 'DRS1', 'DCT2', 'DRS2', 'DCT3', 'DRS3']

for river_segment in river_segments:
    dff = dff_melted.loc[dff_melted['catchment']==river_segment]
    graph_title = river_segment + ' weekly flow' + ' -Reference Scenario'
    fig = px.line(dff, x='week', y='value', color='model',
                  labels={"value": "River discharge (m3/sec)"}, title=graph_title,
                  facet_col_spacing=0.05)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    #You can change the image extension to *.png if you want or keep it as pdf (for high resolution)
    output_folder = os.path.join('processed_data','smhi_ref_graphs', 'Weekly')
    os.makedirs(output_folder, exist_ok = True)

    #fig.write_image('processed_data/smhi_flow_graphs/REF/Weekly/{}.png'.format(graph_title))
    #fig.show()
   

# Part 2: Climate change scenarios

### 2.1 Calculating the average flow for each rcp scenario. 

In [19]:
folder = os.path.join('..','Data', 'smhi_data')
scenario_path = os.path.join(folder ,'cc')

In [20]:
for filepath in glob.iglob(os.path.join(scenario_path, '*.xls')):
    #print(filepath)
    df=pd.read_excel(filepath , sheet_name='E-HYPE312')
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month']=pd.DatetimeIndex(df['Date']).month
    df['week']=pd.DatetimeIndex(df['Date']).week
    df['day']=pd.DatetimeIndex(df['Date']).day
    df=df[(df['year']>=2020) & (df['year']<=2055)]
    i=filepath[21:26]
    df['catchment']= i
    
    # generating average flow for each rcp by taking the mean of different models outputs:
    df2=df.copy()
    df2=df.groupby(['year','week']).mean().round(3).reset_index()
    df2['catchment']= i
    
    df2['RCP26']=df2[['CSC_REMO2009_MPI-ESM-LR_rcp26','SMHI_RCA4_EC-EARTH_rcp26']].mean(axis=1)
    df2['RCP45']=df2[['CSC_REMO2009_MPI-ESM-LR_rcp45','KNMI_RACMO22E_EC-EARTH_rcp45','SMHI_RCA4_EC-EARTH_rcp45','SMHI_RCA4_HadGEM2-ES_rcp45','IPSL-IPSL-CM5A-MR_rcp45' ]].mean(axis=1)
    df2['RCP85']=df2[['CSC_REMO2009_MPI-ESM-LR_rcp85','KNMI_RACMO22E_EC-EARTH_rcp85','SMHI_RCA4_EC-EARTH_rcp85','SMHI_RCA4_HadGEM2-ES_rcp85']].mean(axis=1)
    df2=df2.drop(columns=['month', 'day','CSC_REMO2009_MPI-ESM-LR_rcp26',
           'CSC_REMO2009_MPI-ESM-LR_rcp45', 'CSC_REMO2009_MPI-ESM-LR_rcp85',
           'IPSL-IPSL-CM5A-MR_rcp45', 'KNMI_RACMO22E_EC-EARTH_rcp45',
           'KNMI_RACMO22E_EC-EARTH_rcp85', 'SMHI_RCA4_EC-EARTH_rcp26',
           'SMHI_RCA4_EC-EARTH_rcp45', 'SMHI_RCA4_EC-EARTH_rcp85',
           'SMHI_RCA4_HadGEM2-ES_rcp45', 'SMHI_RCA4_HadGEM2-ES_rcp85'])

   
    #filtering by rcp and saving the files (Note: the missing catchments are not added yet)
    df3 = df2[['catchment','year','week','RCP26']]
    df3['scenario']='RCP26'
    df3['model']='e_hype'
    df3.rename({'RCP26':'value'}, axis=1, inplace=True)
    rcp26folder = os.path.join('processed_data','smhi_cc_data','rcp26')
    os.makedirs(rcp26folder, exist_ok = True)
    df3.to_csv(os.path.join(rcp26folder,'rcp26_'+ i + '.csv'))
    
    
    df4 = df2[['catchment','year','week','RCP45']]
    df4['scenario']='RCP45'
    df4['model']='e_hype'
    df4.rename({'RCP45':'value'}, axis=1, inplace=True)
    rcp45folder = os.path.join('processed_data','smhi_cc_data','rcp45')
    os.makedirs(rcp45folder, exist_ok = True)
    df4.to_csv(os.path.join(rcp45folder,'rcp45_'+ i + '.csv'))
    
    df5 = df2[['catchment','year','week','RCP85']]
    df5['scenario']='RCP85'
    df5['model']='e_hype'
    df5.rename({'RCP85':'value'}, axis=1, inplace=True)
    rcp85folder = os.path.join('processed_data','smhi_cc_data', 'rcp85')
    os.makedirs(rcp85folder, exist_ok = True)
    df5.to_csv(os.path.join(rcp85folder,'rcp85_'+ i + '.csv'))
    
    #combining the flow for all rcps:
    df6 = df3.append([df4,df5])
    df6 = df6[['model','catchment','scenario','year','week','value']]
    allfolder = os.path.join('processed_data','smhi_cc_data','allrcps')
    os.makedirs(allfolder, exist_ok = True)
    df6.to_csv(os.path.join(allfolder,'all_'+ i + '.csv'))
    
    #print(i)
    

<ipython-input-20-de601523eec7>:6: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)

<ipython-input-20-de601523eec7>:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-20-de601523eec7>:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-20-de601523eec7>:47: SettingWith

<ipython-input-20-de601523eec7>:6: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)

<ipython-input-20-de601523eec7>:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-20-de601523eec7>:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-20-de601523eec7>:47: SettingWith

### 2.2 Restructuring the river segments and introducing the missing catchments under each rcp

In [21]:
# This merge will add the missing cathcments and will save merge all catchments under each rcp
# Make sure that you delete the old files, otherwise it might merge old runs as well.
# The result is one rcp file for all catchments. 

rcps = ['rcp26', 'rcp45', 'rcp85']

for rcp in rcps:
    joined_files = os.path.join('processed_data','smhi_cc_data', rcp, "rcp*.csv")

    # A list of all joined files is returned
    joined_list = glob.glob(joined_files)

    # The files are joined
    df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

    df2 = pd.pivot_table(df, values='value', 
        index=[ 'model','scenario', 'year', 'week'], 
        columns=['catchment'], aggfunc=np.sum).round(3).reset_index()

    df2.rename({
    #'ohrid':'ohrid_ct',  #ohrid lake outflow
    #'globo':'BDCT1',      #Inflow from Golbo catchment
    'globo':'BDRS1',      #Inflow from Golbo catchment
    'spilj':'BDRS2',      #Inflow in river segment before spilje
    'black':'BDRS3',      #Inflow in the river segment after spilje and before white drin and skavica
    'white':'WDRS1',      #Inflow from white drin
    'fierz':'DRS1',       #Inflow in river segment before fierza HPP
    'koman':'DRS2',       #Inflow in river segment before koman HPP
    'vaudj':'DRS3'        #Inflow in river segment before Vau i dejes HPP
    }, axis=1, inplace=True)

    #df2[['ohrid', 'BDCT1', 'BDRS1']]
#     df2['BDRS1'] = df2['ohrid'] + df2['BDCT1']  #Inflow in river segment before Globocica
#     df2['BDCT2'] = df2['BDRS2'] - df2['BDRS1']  #Inflow from catchment after Globocica and before Spilje
#     df2['BDCT3'] = df2['BDRS3'] - df2['BDRS2']  #Inflow from catchment after Spilje and before white drin 
#     df2['DCT1'] = df2['DRS1'] - (df2['BDRS3']+df2['WDRS1'])  #Inflow from catchment before Fierza 
#     df2['DCT2'] = df2['DRS2'] - df2['DRS1']                  #Inflow from catchment before Koman
#     df2['DCT3'] = df2['DRS3'] - df2['DRS2']                  #Inflow from catchment before vau i dejes

    years = df2['year']
    for year in years:
        #df2['BDRS1'] = df2['ohrid']+df2['BDCT1']  #Inflow in river segment before Globocica
        df2.loc[df2['BDRS1']>df2['ohrid'], 'BDCT1'] = df2['BDRS1']- df2['ohrid']
        df2.loc[df2['BDRS1']<=df2['ohrid'], 'BDCT1'] = df2['BDCT1'].min()
        

        #Inflow from catchment after Globocica and before Spilje
        df2.loc[df2['BDRS2']>df2['BDRS1'], 'BDCT2'] = df2['BDRS2']-df2['BDRS1']
        df2.loc[df2['BDRS2']<=df2['BDRS1'], 'BDCT2'] = df2['BDCT2'].min()

        #Inflow from catchment after Spilje and before white drin 
        df2.loc[df2['BDRS3']>df2['BDRS2'], 'BDCT3'] = df2['BDRS3']-df2['BDRS2']
        df2.loc[df2['BDRS3']<=df2['BDRS2'], 'BDCT3'] = df2['BDCT3'].min()


        #Inflow from catchment before Fierza 
        df2.loc[df2['DRS1']>(df2['BDRS3']+df2['WDRS1']), 'DCT1'] = df2['DRS1']-(df2['BDRS3']+df2['WDRS1'])
        df2.loc[df2['DRS1']<=(df2['BDRS3']+df2['WDRS1']), 'DCT1'] = df2['WDRS1'].min()

        #Inflow from catchment before Koman
        df2.loc[df2['DRS2']>df2['DRS1'], 'DCT2'] = df2['DRS2']-df2['DRS1']
        df2.loc[df2['DRS2']<=df2['DRS1'], 'DCT2'] = df2['DCT2'].min()

        #Inflow from catchment before vau i dejes
        df2.loc[df2['DRS3']>df2['DRS2'], 'DCT3'] = df2['DRS3']-df2['DRS2']
        df2.loc[df2['DRS3']<=df2['DRS2'], 'DCT3'] = df2['DCT3'].min()

    
    df2 = df2[['model','scenario', 'year', 'week','ohrid', 'BDCT1','BDRS1','BDCT2', 'BDRS2','BDCT3', 'BDRS3', 'WDRS1', 'DCT1','DRS1','DCT2', 'DRS2','DCT3','DRS3']]

    dff = df2.copy()
    dff_melted = dff.melt(id_vars=['model','scenario','year','week'], var_name='catchment',value_name='value',ignore_index=True)

    dff_melted = dff_melted[['model','scenario','year','catchment','week','value']]

    results_folder = os.path.join('processed_data','smhi_cc_data', 'adding_missing_catchments')
    os.makedirs(results_folder, exist_ok = True)
    dff_melted.to_csv(os.path.join(results_folder, 'adding_missing_catchments_' + rcp + '.csv'))


### 2.3 Merging the files for the three rcps into one final dataset

In [22]:
# In this step we will merge the files for the three rcps in one final dataset

#for rcp in rcps:
joined_files = os.path.join('processed_data','smhi_cc_data', 'adding_missing_catchments', "adding_missing_catchments_*.csv")
#print(joined_files)

# A list of all joined files is returned
joined_list = glob.glob(joined_files)
#print(joined_list)

# The files are joined
df_all = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df_all.drop(['Unnamed: 0'], axis=1, inplace=True)

results_folder = os.path.join('processed_data','smhi_cc_data')
os.makedirs(results_folder, exist_ok=True)
df_all.to_csv(os.path.join(results_folder, ('ehype_cc_allcatchments_allrcps_d.csv')),index= False)

# Part 3: Calculating river segments Capacity Factor (CF), Residual capacity and lower activity limit: 

## The capacity factor of each river segment is calculated based on:

CF = (the projected average weekly flow) / (the historical max flow) 

Q: should we use the max historical value? or the max monthly value?


## The residual capacity is calculated based on: 

The mean (max?) river flow for each catchment in each year * (3600 sec)*(8760 days) and divide by 1000000 to get the capacity in MCM.

## The lower Activity limit is based on: 
lowerlimit = the sum of the weekly flows in each year (average of several years). (this is the first approach)

comment: 
This might result in over production from river segments.

try changing this to: lowerlimit = weeklyflow.min() * 52 weeks. 
This means that we force the river segment to at least produce to the min flow. 


In [2]:
# Reading the input files: 
input_folder = os.path.join('processed_data')
df_ref = pd.read_csv(os.path.join(input_folder, 'smhi_ref_data', 'ehype_ref_weeklyavg_allcatchments_b20220211.csv'))
df_cc = pd.read_csv(os.path.join(input_folder, 'smhi_cc_data','ehype_cc_allcatchments_allrcps_d.csv'))


In [3]:
# Combined script:
# This scrip will take the input files per scenario (ref, cc) and calculate the capacity factor, residual capacity and lower activity limit
# It will then save the files for each scenario, catchment, rcp and parameter:


scenarios = ['REF', 'CC']

for scenario in scenarios:
    if scenario == 'REF':
        df = df_ref
    else:
        df = df_cc
    
        
# Deleting the segments that will not be used as inputs to osemosys:
#     df = df.loc[(df['catchment']!='BDRS1')&(df['catchment']!='BDRS2')&(df['catchment']!='BDRS3')
#       &(df['catchment']!='DRS1')&(df['catchment']!='DRS2')&(df['catchment']!='DRS3')]
    
    # Renaming the river segments to match osemosys structure:
    df.set_index('catchment', inplace=True)

    df.rename({'ohrid':'MKCWTLK0BD',
               'DCT1':'ALCWTCT1DD',
               'BDCT1':'MKCWTCT1BD',
               'DCT2':'ALCWTCT2DD',
               'Orhid':'MKCWTLK0BD',
               'BDCT2':'MKCWTCT2BD',
               'BDCT3':'MKCWTCT3DD',
               'DCT3':'ALCWTCT3DD',
               'WDRS1':'XKCWTRS1WD',
               'BDRS1':'MKCWTRS1BD',
               'BDRS2':'MKCWTRS2BD',
               'BDRS3':'MKCWTRS3DD',
               'DRS1':'ALCWTRS1DD',
               'DRS2':'ALCWTRS2DD',
               'DRS3':'ALCWTRS3DD'
              }, axis=0, inplace=True)

    df.reset_index(inplace=True)
    
    max_value = df_ref.loc[df_ref['week']!=53]
    max_value = df_ref.groupby(['catchment'])['value'].max()
    max_value.rename({'ohrid':'MKCWTLK0BD',
                   'DCT1':'ALCWTCT1DD',
                   'BDCT1':'MKCWTCT1BD',
                   'DCT2':'ALCWTCT2DD',
                   'Orhid':'MKCWTLK0BD',
                   'BDCT2':'MKCWTCT2BD',
                   'BDCT3':'MKCWTCT3DD',
                   'DCT3':'ALCWTCT3DD',
                   'WDRS1':'XKCWTRS1WD',
                    'BDRS1':'MKCWTRS1BD',
                   'BDRS2':'MKCWTRS2BD',
                   'BDRS3':'MKCWTRS3DD',
                   'DRS1':'ALCWTRS1DD',
                   'DRS2':'ALCWTRS2DD',
                   'DRS3':'ALCWTRS3DD'
            }, axis=0, inplace=True)
    #max_value.drop(['BDRS1', 'BDRS2', 'BDRS3','DRS1', 'DRS2', 'DRS3'], inplace=True) 
    
    mean_value = df_ref.loc[df_ref['week']!=53]
    mean_value = df_ref.groupby(['catchment'])['value'].mean()
    mean_value.rename({'ohrid':'MKCWTLK0BD',
                   'DCT1':'ALCWTCT1DD',
                   'BDCT1':'MKCWTCT1BD',
                   'DCT2':'ALCWTCT2DD',
                   'Orhid':'MKCWTLK0BD',
                   'BDCT2':'MKCWTCT2BD',
                   'BDCT3':'MKCWTCT3DD',
                   'DCT3':'ALCWTCT3DD',
                   'WDRS1':'XKCWTRS1WD',
                    'BDRS1':'MKCWTRS1BD',
                   'BDRS2':'MKCWTRS2BD',
                   'BDRS3':'MKCWTRS3DD',
                   'DRS1':'ALCWTRS1DD',
                   'DRS2':'ALCWTRS2DD',
                   'DRS3':'ALCWTRS3DD'
            }, axis=0, inplace=True)
    #mean_value.drop(['BDRS1', 'BDRS2', 'BDRS3','DRS1', 'DRS2', 'DRS3'], inplace=True) 
    
    
    # Merge the projected values with the historical max for each river segment and calculate the CF:
    res_cap = df.groupby(['catchment', 'scenario', 'year'])['value'].max()*(3600*8760/1000000)
    lowlimit = df.groupby(['catchment','scenario','year'])['value'].min()*52
    
    
    df2 = pd.merge(df, max_value, on=['catchment'])
    df2.rename({'value_x':'proj_weekly_flow' ,
                     'value_y':'hist_max_flow'}, axis=1, inplace=True)
    df2['cf'] = df2['proj_weekly_flow']/df2['hist_max_flow']
    
    
    #res capacity calculations: 

#     res_cap = df2.groupby(['catchment', 'scenario', 'year'])['proj_weekly_flow'].mean()*(3600*8760/1000000)

    df3 = pd.merge(df2, res_cap, on=['catchment','scenario','year'])
    
    df3.rename({'value':'res_cap'
                }, axis=1, inplace=True)
    df3=df3.drop_duplicates()
    
#     #Lower Activity calculations: 
    #lowlimit = df3.groupby(['catchment','scenario','year'])['proj_weekly_flow'].sum()

    df4 = pd.merge(df3, lowlimit, on=['catchment','scenario','year'])
    df4.rename({'value':'loweractivitylimit'
                }, axis=1, inplace=True)
    df4=df4.drop_duplicates()


###### SAVING THE OUTPUT FILES: #############

    catchments = df4['catchment'].unique()
    
    if scenario == 'REF':
        rcps = ['ref']
    else:
        rcps = ['RCP26','RCP45','RCP85']
        
        
    for catchment in catchments:
        for rcp in rcps:
            output_folder = os.path.join('processed_data', 'final_outputs', scenario,'capacity_factors')
            os.makedirs(output_folder, exist_ok = True)
            dfcf=  df4.loc[df4['catchment']==catchment]
            dfcf = dfcf.loc[dfcf['scenario']==rcp]
            dfcf = dfcf.loc[dfcf['week']!=53]
            dfcf = pd.pivot_table(dfcf, values='cf',
                                 index=['scenario','catchment','week'],
                                 columns=['year'], aggfunc=np.mean).round(3).reset_index()
            
            dfcf.to_csv(os.path.join(output_folder, 'Capacity_factors_' + rcp + '_' + catchment + '.csv'),index=False)
    
    
    for rcp in rcps:
        output_folder = os.path.join('processed_data', 'final_outputs', scenario,'res_capacity')
        os.makedirs(output_folder, exist_ok = True)
        #dff= df4.loc[df4['catchment']==catchment]
        dfrc = df4.loc[df4['scenario']==rcp]
        dfrc = dfrc.loc[dfrc['week']!=53]
        dfrc = pd.pivot_table(dfrc, values='res_cap',
                            index=['scenario', 'year'],
                            columns=['catchment'], aggfunc=np.mean).round(3).reset_index()
        #Adjusting the residual capacity values:
        #Globocica
        dfrc['MKCWTLK0BD']=dfrc.MKCWTLK0BD
        dfrc['MKCWTCT1BD']=dfrc.MKCWTCT1BD
        dfrc['MKCWTRS1BD']=dfrc['MKCWTLK0BD']+dfrc['MKCWTCT1BD']
        #dfrc['MKCWTSP1BD'] = 

        #Shpilje
        dfrc['MKCWTCT2BD']=dfrc.MKCWTCT2BD
        dfrc['MKCWTRS2BD']=dfrc['MKCWTRS1BD']+dfrc['MKCWTCT2BD']
        #dfrc['MKCWTSP2BD'] =

        #Skavica
        dfrc['MKCWTCT3DD']=dfrc.MKCWTCT3DD
        dfrc['MKCWTRS3DD']=dfrc['MKCWTRS2BD']+dfrc['MKCWTCT3DD']
        #dfrc['ALCWTSP3DD'] =


        #White_Drin
        dfrc['XKCWTRS1WD']=dfrc.XKCWTRS1WD

        #Fierza
        dfrc['ALCWTCT1DD']=dfrc.ALCWTCT1DD
        dfrc['ALCWTRS1DD']=dfrc['MKCWTRS3DD']+ dfrc['XKCWTRS1WD']+dfrc['ALCWTCT1DD']
        #dfrc['ALCWTSP4DD'] =

        #Koman
        dfrc['ALCWTCT2DD']=dfrc.ALCWTCT2DD
        dfrc['ALCWTRS2DD']=dfrc['ALCWTRS1DD']+ dfrc['ALCWTCT2DD']
        #dfrc['ALCWTSP5DD'] =

        #Vau_dejas
        dfrc['ALCWTCT3DD']=dfrc.ALCWTCT3DD
        dfrc['ALCWTRS3DD']=dfrc['ALCWTRS2DD']+ dfrc['ALCWTCT3DD']
        #dfrc['ALCWTSP6DD'] =

        #Drin_outflow
        #dfrc['ALCWTCT4DD']=dfrc.ALCWTCT4DD
       
    
        dfrc = dfrc.T.reset_index()
        dfrc.columns = dfrc.iloc[1]
        dfrc = dfrc.drop(labels=[0,1], axis=0)
        
#         for i in range(20,56):
#             if scenario =='REF':
#                 dfrc['20{}'.format(i)] = dfrc[2020]
#             else:
#                 dfrc['20'{}'.format(i)]
#         #dfrc.drop(2020, axis=1, inplace=True)
        dfrc.to_csv(os.path.join(output_folder, 'res_capacity_' + rcp + '.csv'),index=False)
            
    
    for rcp in rcps:
        output_folder = os.path.join('processed_data', 'final_outputs', scenario,'loweractivity')
        os.makedirs(output_folder, exist_ok = True)
        #dff= df4.loc[df4['catchment']==catchment]
        dfla = df4.loc[df4['scenario']==rcp]
        dfla = dfla.loc[dfla['week']!=53]
        dfla = pd.pivot_table(dfla, values='loweractivitylimit',
                             index=['scenario','catchment'],
                             columns=['year'], aggfunc=np.mean).round(3).reset_index()

        dfla.to_csv(os.path.join(output_folder, 'loweractivity_' + rcp + '.csv'),index=False)

In [9]:
# Post processing of the CF dataframes for the REF SCENARIO ONLY:


scenarios = ['REF']

for scenario in scenarios:
    input_folder = os.path.join('processed_data', 'final_outputs', scenario,'capacity_factors')

    for filepath in glob.iglob(os.path.join(input_folder, ('*.csv'))):
        dfcf2=pd.read_csv(filepath)
        name = filepath[71:81]  #check if the naming is correct, else adjust this
        
#        print(filepath)

        for i in range(20,56):
            if scenario =='REF':
                dfcf2['20{}'.format(i)] = dfcf2['2020']
            else:
                dfcf2['20{}'.format(i)]
            output_folder = os.path.join('processed_data', 'final_outputs', scenario,'capacity_factors2')
            os.makedirs(output_folder, exist_ok= True)
            dfcf2.to_csv(os.path.join(output_folder,('CapacityFactor2_'+str(name)+'.csv')))

# END